# Frontloading

# Phase 2 vs. Phase 2

Following protocol from Gurr et al., “Acclimatory Gene Expression of Primed Clams Enhances Robustness to Elevated pCO2.” - who was originally inspired by Barshis et al., “Genomic Basis for Coral Resilience to Climate Change.”

[Gurr GitHub script](https://github.com/SamGurr/Pgenerosa_OA_TagSeq/blob/main/TagSeq/Analysis/Scripts/DE_frontloading.R)

## 0. load libraries

In [2]:
library(tidyverse)

## 1. read CSVs

In [10]:
# get list of files
files <- list.files(
    path = '/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/DEGs',
    pattern = '\\.csv$',
    full.names = TRUE
    )

# read each csv
deg_list <- lapply(files, read.csv)

# name list elements after the files
names(deg_list) <- gsub('DEG_','',tools::file_path_sans_ext(basename(files)))

# check things worked
names(deg_list)
head(deg_list$bb_cc)

[1] "bb_cc" "bc_bb" "bc_cb" "bc_cc" "bc_hc" "bc_wc" "bh_hh" "bw_ww" "cb_bb"
[10] "cb_bc" "cb_cc" "cb_ch" "cb_cw" "ch_cc" "ch_hc" "cw_cc" "cw_ch" "cw_wc"
[19] "hb_bb" "hb_bh" "hb_cb" "hc_cc" "hc_ch" "hc_hh" "hh_cc" "hh_ch" "wb_bb"
[28] "wb_bw" "wb_cb" "wc_cc" "wc_cw" "wc_hc" "wc_ww" "wh_hw" "ww_cc" "ww_cw"

,X,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,svalue,pair,DEG_group
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,LOC111134671,613.020976,1.705185,1.8602511,-3.890429,1.000673e-04,0.023871608,0.123867402,CC vs. BB,BB
2,LOC111120923,146.529937,-5.243692,4.1847160,3.947822,7.886530e-05,0.020048777,0.059178297,CC vs. BB,CC
3,LOC111136853,86.689266,-1.963764,2.6599338,3.713204,2.046516e-04,0.040461455,0.170868504,CC vs. BB,CC
4,LOC111120760,103.711700,2.616076,0.9408944,-4.651563,3.294284e-06,0.001389420,0.006801697,CC vs. BB,BB
5,LOC111125595,123.169520,17.307451,9.1757285,-4.195594,2.721571e-05,0.008903943,0.022714995,CC vs. BB,BB
6,LOC111114504,6.398528,7.268181,7.7009243,-3.710223,2.070769e-04,0.040461455,0.119596502,CC vs. BB,BB


## 2. Frontloading!

### Step 1: pull up-regulated genes in oysters first exposed to control conditions
The first step in identifying frontloading is to pull genes that were up-regulated in oysters that were first exposed to control conditions, and then were exposed to stress (CW, CH, CB vs. CC)

(also going to do these steps simultaneously for warm, hypoxia, and both, but just focusing on frontloading within these stressors - not thinking about interactions of stressors yet)

In [13]:
# warm up-regulated
cw_cc.UP <- deg_list$cw_cc %>% filter(log2FoldChange > 0)
dim(cw_cc.UP)

# hypoxia up-regulated
ch_cc.UP <- deg_list$ch_cc %>% filter(log2FoldChange > 0)
dim(ch_cc.UP)

# both up-regulated
cb_cc.UP <- deg_list$cb_cc %>% filter(log2FoldChange > 0)
dim(cb_cc.UP)

[1] 71 10

[1] 72 10

[1] 80 10

### Step 2: 

In [21]:
metaData <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/sample_metaData.csv') %>%
mutate(All_treatment = factor(paste0(Phase1_treatment, '_', Phase2_treatment))) %>% # add column for p1 and p2 treatment combined, make treatment groups a factor
select(Sample, Phase1_treatment, Phase2_treatment, All_treatment) %>% # grab only the needed columns 
filter(!is.na(Phase2_treatment)) # remove phase 1 samples

head(metaData)

,Sample,Phase1_treatment,Phase2_treatment,All_treatment
,<chr>,<chr>,<chr>,<fct>
1,B1_B1_O01,both,both,both_both
2,B1_W5_O50,both,warm,both_warm
3,B2_B5_O51,both,both,both_both
4,B2_C4_O40,both,control,both_control
5,B3_B4_O41,both,both,both_both
6,B3_C3_O30,both,control,both_control


In [22]:
head(cb_cc.UP)

,X,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,svalue,pair,DEG_group
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,LOC111114504,6.398528,17.730172,4.643629,4.614806,3.934628e-06,1.064270e-03,1.285587e-03,CC vs. CB,CB
2,LOC111112854,55.739122,22.946058,3.273051,7.130488,1.000136e-12,8.379680e-10,1.035453e-09,CC vs. CB,CB
3,LOC111099906,5.155636,14.336568,4.216457,4.484528,7.307536e-06,1.793061e-03,3.231100e-03,CC vs. CB,CB
4,LOC111110967,398.914414,1.542606,1.744475,3.881107,1.039823e-04,1.768316e-02,1.071955e-01,CC vs. CB,CB
5,LOC111119065,7.511249,17.605222,3.059877,5.842224,5.150849e-09,2.527742e-06,5.343031e-06,CC vs. CB,CB
6,LOC111121290,13.027084,2.902724,3.093456,3.779762,1.569783e-04,2.414612e-02,7.949648e-02,CC vs. CB,CB
